In [4]:
#generate pb for VESPCN
import os, time
import tensorflow as tf
import imageio
from subpixel import *
from utils import *

%reload_ext autoreload
%autoreload 2

W = 720
H = 480
scale = 3
#mode = "RGB"
mode = "YCbCr"
channel = 3
num_input_frames = 3
xx = True
if mode == "YCbCr":
    channel = 1
    xx = False
model_name = "VESPCN-"+str(mode)+"-x"+str(scale)+"-input"+str(num_input_frames)
pb_name = "VESPCN-"+str(mode)+"-"+str(H)+"-x"+str(scale)+"-input"+str(num_input_frames)+".pb"

W1 = 0
W2 = 0
W3 = 0
W4 = 0
W5 = 0
W6 = 0
B1 = 0
B2 = 0
B3 = 0
B4 = 0
B5 = 0
B6 = 0

tf.reset_default_graph()

sess = tf.Session()
with sess.as_default():
    LR = tf.placeholder(tf.float32, [1, H, W, channel, num_input_frames], name= 'input')
    LR = tf.reshape(LR, [LR.shape[0], LR.shape[1], LR.shape[2], LR.shape[3]* LR.shape[4]])
    feature_tmp = tf.layers.conv2d(LR, 24, 3, strides = 1, padding = 'SAME', name = 'CONV_1',
                                kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.layers.conv2d(feature_tmp, 24, 3, strides = 1, padding = 'SAME', name = 'CONV_2',
                                kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.layers.conv2d(feature_tmp, 24, 3, strides = 1, padding = 'SAME', name = 'CONV_3',
                                kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.layers.conv2d(feature_tmp, 24, 3, strides = 1, padding = 'SAME', name = 'CONV_4',
                                kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_out = tf.layers.conv2d(feature_tmp, channel*scale*scale, 3, strides = 1, padding = 'SAME', 
                   name = 'CONV_5', kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_out = PS(feature_out, scale, color=xx)
    feature_out = tf.layers.conv2d(feature_out, channel, 1, strides = 1, padding = 'SAME', 
                        name = 'CONV_OUT', kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    init_op = tf.global_variables_initializer() 
    
    saver = tf.train.Saver()
    saver.restore(sess, os.path.join("./checkpoint/VSR/CDVL", model_name))

    vv = [v for v in tf.global_variables()]
    print("variable list: ", vv)
    W1 = [v for v in tf.global_variables() if v.name == "CONV_1/kernel:0"][0].eval()
    W2 = [v for v in tf.global_variables() if v.name == "CONV_2/kernel:0"][0].eval()
    W3 = [v for v in tf.global_variables() if v.name == "CONV_3/kernel:0"][0].eval()
    W4 = [v for v in tf.global_variables() if v.name == "CONV_4/kernel:0"][0].eval()
    W5 = [v for v in tf.global_variables() if v.name == "CONV_5/kernel:0"][0].eval()
    W6 = [v for v in tf.global_variables() if v.name == "CONV_OUT/kernel:0"][0].eval()
    B1 = [v for v in tf.global_variables() if v.name == "CONV_1/bias:0"][0].eval()
    B2 = [v for v in tf.global_variables() if v.name == "CONV_2/bias:0"][0].eval()
    B3 = [v for v in tf.global_variables() if v.name == "CONV_3/bias:0"][0].eval()
    B4 = [v for v in tf.global_variables() if v.name == "CONV_4/bias:0"][0].eval()
    B5 = [v for v in tf.global_variables() if v.name == "CONV_5/bias:0"][0].eval()
    B6 = [v for v in tf.global_variables() if v.name == "CONV_OUT/bias:0"][0].eval()

tf.reset_default_graph()
sess2 = tf.Session()
print("graph reset!")
with sess2.as_default():
    LR = tf.placeholder(tf.float32, [1, H, W, channel, num_input_frames], name= 'input')
    LR_in = tf.reshape(LR, [LR.shape[0], LR.shape[1], LR.shape[2], LR.shape[3]* LR.shape[4]])
    feature_tmp = tf.nn.conv2d(LR_in, W1, strides=[1, 1, 1, 1], padding='SAME')
    feature_tmp = tf.nn.bias_add(feature_tmp, B1)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.nn.conv2d(feature_tmp , W2, strides=[1, 1, 1, 1], padding='SAME')
    feature_tmp = tf.nn.bias_add(feature_tmp, B2)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.nn.conv2d(feature_tmp , W3, strides=[1, 1, 1, 1], padding='SAME')
    feature_tmp = tf.nn.bias_add(feature_tmp, B3)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.nn.conv2d(feature_tmp , W4, strides=[1, 1, 1, 1], padding='SAME')
    feature_tmp = tf.nn.bias_add(feature_tmp, B4)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.nn.conv2d(feature_tmp , W5, strides=[1, 1, 1, 1], padding='SAME')
    feature_tmp = tf.nn.bias_add(feature_tmp, B5)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_out = PS(feature_tmp, scale, color=xx)
    output_HR = tf.nn.conv2d(feature_out, W6, strides=[1, 1, 1, 1], padding='SAME')
    output_HR = tf.nn.bias_add(output_HR, B6)
    output_HR = tf.clip_by_value(output_HR, 0, 255, name = 'output')
    #output_HR = tf.add(tf.multiply(tf.add(tf.nn.tanh(feature_out),1),(235-16)/2), 16, name = 'output') #Y range is in 16-235
    tf.train.write_graph(sess2.graph_def, "./checkpoint/pb", pb_name, as_text=False)

    vv = [v for v in tf.global_variables()]
    print("Drew new graph! variable list: ", vv)

INFO:tensorflow:Restoring parameters from ./checkpoint/VSR/CDVL/VESPCN-YCbCr-x3-input3
variable list:  [<tf.Variable 'CONV_1/kernel:0' shape=(3, 3, 3, 24) dtype=float32_ref>, <tf.Variable 'CONV_1/bias:0' shape=(24,) dtype=float32_ref>, <tf.Variable 'CONV_2/kernel:0' shape=(3, 3, 24, 24) dtype=float32_ref>, <tf.Variable 'CONV_2/bias:0' shape=(24,) dtype=float32_ref>, <tf.Variable 'CONV_3/kernel:0' shape=(3, 3, 24, 24) dtype=float32_ref>, <tf.Variable 'CONV_3/bias:0' shape=(24,) dtype=float32_ref>, <tf.Variable 'CONV_4/kernel:0' shape=(3, 3, 24, 24) dtype=float32_ref>, <tf.Variable 'CONV_4/bias:0' shape=(24,) dtype=float32_ref>, <tf.Variable 'CONV_5/kernel:0' shape=(3, 3, 24, 9) dtype=float32_ref>, <tf.Variable 'CONV_5/bias:0' shape=(9,) dtype=float32_ref>, <tf.Variable 'CONV_OUT/kernel:0' shape=(1, 1, 1, 1) dtype=float32_ref>, <tf.Variable 'CONV_OUT/bias:0' shape=(1,) dtype=float32_ref>]
graph reset!
Drew new graph! variable list:  []


In [22]:
img0 = scipy.misc.imread(os.path.join("/home/johnyi/deeplearning/research/VSR_Datasets/test/vid4/walk/001.png"), mode = "YCbCr")
img1 = scipy.misc.imread(os.path.join("/home/johnyi/deeplearning/research/VSR_Datasets/test/vid4/walk/002.png"), mode = "YCbCr")
img2 = scipy.misc.imread(os.path.join("/home/johnyi/deeplearning/research/VSR_Datasets/test/vid4/walk/003.png"), mode = "YCbCr")
Y0 = np.split(img0, 3, axis=2)[0]
Y1 = np.split(img1, 3, axis=2)[0]
Y2 = np.split(img2, 3, axis=2)[0]
Cb = np.split(img1, 3, axis=2)[1]
Cr = np.split(img2, 3, axis=2)[2]
img = np.zeros([1, H, W, 1, 3])
img[0, :,:,:, 0] = Y0
img[0, :,:,:, 1] = Y1
img[0, :,:,:, 2] = Y2
start_time = time.time()
[out] = sess2.run([output_HR], feed_dict={LR: img})
print("runtime: %.6f" %(time.time()-start_time))
out = np.round(out).astype("uint8")
print(out.shape)
save_ycbcr_img(out[0], Cb, Cr, 3, "test.png")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


runtime: 0.011918
(1, 1440, 2160, 1)


0

In [37]:
# Test loading pb file (YCbCr)
from tensorflow.core.framework import graph_pb2
import os
import time
import numpy as np

W = 256
H = 256
scale = 3
mode = "YCbCr"
channel = 3
xx = True
if mode == "YCbCr":
    channel = 1
    xx = False

pb_name = "ESPCN-"+str(mode)+"-"+str(H)+"-x"+str(scale)+".pb"
tf.reset_default_graph()
sess = tf.Session()
with sess.as_default():        
    with open(os.path.join('./checkpoint/pb/', pb_name), 'rb') as f:
        input_LR = tf.placeholder(tf.float32, [None, H, W, channel], name='input')
        #graph_def = tf.GraphDef()
        graph_def = graph_pb2.GraphDef()
        graph_def.ParseFromString(f.read())
        output = tf.import_graph_def(graph_def, input_map={'input:0': input_LR}, return_elements=['output:0'])
        output = output[0]
        #x = sess.graph.get_operations()
        #print("ops:",x)
        print(output)
        
        img = load_image(os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/Set5/baby.png"), "YCbCr")
        img = doresize(img, [H, W])
        Y_bicubic = np.split(imresize(img,[scale*H, scale*W], interp='bicubic'),3,axis=2)[0]
        Y_LR = np.split(img,3, axis=2)[0]
        Cb_LR = np.split(img,3, axis=2)[1]
        Cr_LR = np.split(img,3, axis=2)[2]
        start_time = time.time()
        [out] = sess.run([output], feed_dict={input_LR: [Y_LR]})
        print("runtime: %.6f" %(time.time()-start_time))
        out = np.round(out).astype("uint8")
        print("out size:",out[0].shape, np.min(out[0]), np.max(out[0]))
        print("out size:",Cb_LR[0].shape, np.min(Cb_LR[0]), np.max(Cb_LR[0]))
        print("out size:",Cr_LR[0].shape, np.min(Cr_LR[0]), np.max(Cr_LR[0]))
        save_ycbcr_img(out[0], Cb_LR, Cr_LR, scale, "./test.png")
        save_ycbcr_img(Y_bicubic, Cb_LR, Cr_LR, scale, "./test_bicubic.png")
        
runtimes = []
for i in range(0,10):
    img = load_image(os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/Set5/baby.png"), "YCbCr")
    img = doresize(img, [H, W])
    Y_bicubic = np.split(imresize(img,[scale*H, scale*W], interp='bicubic'),3,axis=2)[0]
    Y_LR = np.split(img,3, axis=2)[0]
    Cb_LR = np.split(img,3, axis=2)[1]
    Cr_LR = np.split(img,3, axis=2)[2]
    start_time = time.time()
    [out] = sess.run([output], feed_dict={input_LR: [Y_LR]})
    runtime = time.time()-start_time
    print("%.6f" %(runtime))
    runtimes.append(runtime)
    #print("runtime: %.6f" %(time.time()-start_time))
    out = np.round(out).astype("uint8")
    #print("out size:",out[0].shape, np.min(out[0]), np.max(out[0]))
    #print("out size:",Cb_LR[0].shape, np.min(Cb_LR[0]), np.max(Cb_LR[0]))
    #print("out size:",Cr_LR[0].shape, np.min(Cr_LR[0]), np.max(Cr_LR[0]))
print('avg: %.6f' %(np.sum(runtimes)/len(runtimes)))

Tensor("import/output:0", shape=(?, 768, 768, 1), dtype=float32)
runtime: 0.071890
out size: (768, 768, 1) 16 235
out size: (256, 1) 88 129
out size: (256, 1) 125 168


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:52: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


0.002203
0.002260
0.002369
0.002322
0.002267
0.002225
0.002291
0.002165
0.002220
0.002680
avg: 0.002300


In [89]:
img = load_image(os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/Set5/baby.png"), "YCbCr")
img = doresize(img, [H, W])
Y_bicubic = np.split(imresize(img,[scale*H, scale*W], interp='bicubic'),3,axis=2)[0]
Y_LR = np.split(img,3, axis=2)[0]
Cb_LR = np.split(img,3, axis=2)[1]
Cr_LR = np.split(img,3, axis=2)[2]
start_time = time.time()
[out] = sess.run([output_HR], feed_dict={input_LR: [Y_LR]})
print("runtime: %.6f" %(time.time()-start_time))
out = np.round(out).astype("uint8")
print("out size:",out[0].shape, np.min(out[0]), np.max(out[0]))
print("out size:",Cb_LR[0].shape, np.min(Cb_LR[0]), np.max(Cb_LR[0]))
print("out size:",Cr_LR[0].shape, np.min(Cr_LR[0]), np.max(Cr_LR[0]))
#save_ycbcr_img(out[0], Cb_LR, Cr_LR, scale, "./test.png")
#save_ycbcr_img(Y_bicubic, Cb_LR, Cr_LR, scale, "./test_bicubic.png")

runtime: 0.005253
out size: (1134, 1512, 1) 16 235
out size: (504, 1) 89 129
out size: (504, 1) 125 168


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
